## Busca Tabu 

A Busca Tabu é uma meta heurística que integra estruturas de memórias com estratégias de busca local. A ideia básica dessa busca é penalizar movimentos que levem a solução para espaços de busca já visitados, mesmo que isso leve para soluções piores que a solução atual.

In [1]:
from random import sample
import numpy as np


OBJECT_ARRAY = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
WEIGHT_ARRAY = (63, 21, 2, 32, 13, 80, 19, 37, 56, 41, 14, 8, 32, 42, 7)
VALUE_ARRAY = (13, 2, 20, 10, 7, 14, 7, 2, 2, 4, 16, 17, 17, 3, 21)
WEIGHT_LIMIT = 275


class Backpack(object):
    def __init__(self, objects=dict()):
        self.objects = dict()
        self.weight = 0
        self.value = 0

        for obj in objects:
            self.add(obj)

    def add(self, obj):
        self.objects[obj['id']] = obj

        self.weight += obj['weight']
        self.value += obj['value']
    
    def remove(self, obj_id):
        self.weight -= self.objects[obj_id]['weight']
        self.value -= self.objects[obj_id]['value']

        self.objects.pop(obj_id, None)
    
    def representation(self):
        # Binary representation:
        # 0 = not in backpack
        # 1 = in backpack
        representation = np.zeros(len(objects), dtype=int)

        for obj in self.objects:
            representation[obj - 1] = 1

        # Return representation as string
        return "".join(str(num) for num in representation)
    
    def get_neighbours(self, tabu_list):
        tabu_set = set(tabu_list)
        solutions = list()

        # Temporarily randomly generate 10 solutions
        while len(solutions) != 10:
            shuffle_objects = sample(objects, len(objects))
            solution = Backpack(dict())

            for obj in shuffle_objects:
                if solution.weight + obj['weight'] < WEIGHT_LIMIT:
                    solution.add(obj)
            
            if solution.representation not in tabu_set:
                solutions.append(solution)
                    
        return solutions


objects = list()

for id, weight, value in zip(OBJECT_ARRAY, WEIGHT_ARRAY, VALUE_ARRAY):
    new_object = {
        'id': id, 
        'weight': weight, 
        'value': value
    }
    objects.append(new_object)

len(objects)

15

In [2]:
# Get greedy solution to use it as the Termination Criteria
ordered_objects = sorted(objects, 
                         key=lambda obj: obj['value'] / obj['weight'], 
                         reverse=True)

greedy_solution = Backpack()

for obj in ordered_objects:
    if greedy_solution.weight + obj['weight'] <= WEIGHT_LIMIT:
        greedy_solution.add(obj)

print(greedy_solution.representation())
greedy_solution.weight, greedy_solution.value

101111100011101


(270, 142)

In [3]:
# Randomly create initial solution
from random import sample

shuffle_objects = sample(objects, len(objects))
initial_solution = Backpack()

for obj in shuffle_objects:
    if initial_solution.weight + obj['weight'] <= WEIGHT_LIMIT:
        initial_solution.add(obj)

print(initial_solution.representation())
initial_solution.weight, initial_solution.value

111001010011011


(274, 108)

In [4]:
from collections import deque

TABU_TERNURE = 100
MAX_ITERATIONS = 100000


def tabu_search(initial_solution):
    tabu_list = deque()

    iterations = 0
    best_solution = initial_solution
    best_neighbour = initial_solution

    # Termination criteria: find a better solution than greedy or reach max iterations
    while best_solution.value <= greedy_solution.value and iterations < MAX_ITERATIONS:
        neighbouring_solutions = best_neighbour.get_neighbours(tabu_list)
        
        # Get best neighbour based on value
        best_neighbour = max(neighbouring_solutions, 
                            key=lambda solution: solution.value)

        if best_neighbour.value > best_solution.value:
            del best_solution

            print(best_neighbour.representation())
            print(best_neighbour.value)

            best_solution = best_neighbour

        # Update Tabu list
        tabu_list.append(best_neighbour)
        if len(tabu_list) > TABU_TERNURE:
            tabu_list.popleft()
        
        iterations += 1

    return best_solution


best_solution = tabu_search(initial_solution)
print(f"Best cost found: {best_solution.value}")
print(f"Items: {[obj for obj in best_solution.objects]}")

101110100111111
135
111111000011101
137
101111100011101
142
Best cost found: 142
Items: [15, 6, 7, 3, 1, 4, 5, 13, 12, 11]
